<a href="https://colab.research.google.com/github/Santhoshini-Reddy/AIML1/blob/main/AIML5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path='/titanic_train.csv'

In [ ]:
train = pd.read_csv(file_path)
train.tail()

In [ ]:
train.info(verbose=True)

In [ ]:
d=train.describe()
d

In [ ]:
dT=d.T
dT.plot.bar(y='count')
plt.title("Bar plot of the count of numeric features",fontsize=17)
plt.show()

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=train,palette='RdBu_r')

sns.pairplot(train)
plt.show()

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')
plt.show()

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='RdBu_r')
plt.show()

In [ ]:
f_class_survived=train.groupby('Pclass')['Survived'].mean()
f_class_survived=pd.DataFrame(f_class_survived)
f_class_survived
f_class_survived.plot.bar(y='Survived')
plt.title("Survival Rate by Passenger Class",fontsize=17)
plt.show()

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='SibSp',data=train,palette='rainbow')
plt.title("Survival Count Based on Number of Siblings/Spouses",fontsize=17)
plt.show()

In [ ]:
plt.xlabel("Age of the passengers",fontsize=18)
plt.ylabel("Count",fontsize=18)
plt.title("Histogram of the passengers",fontsize=22)
train['Age'].hist()
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
plt.xlabel("Passenger Class",fontsize=18)
plt.ylabel("Age",fontsize=18)
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

In [ ]:
f_class_Age=train.groupby('Pclass')['Age'].mean()
f_class_Age=pd.DataFrame(f_class_Age)
f_class_Age.plot.bar(y='Age')
plt.title("Average Age of  Passenger by Class",fontsize=17)
plt.ylabel("Age(years)",fontsize=17)
plt.xlabel("Passenger class",fontsize=17)
plt.show()

In [ ]:
a=list(f_class_Age['Age'])

def impute_age(cols):
  Age=cols[0]
  Pclass=cols[1]
  if pd.isnull(Age):
    if Pclass==1:
      return a[0]
    elif Pclass==2:
        return a[1]
    else:
        return a[2]
  else:
      return Age

In [ ]:
train['Age']=train[['Age','Pclass']].apply(impute_age,axis=1)
d=train.describe()
dT=d.T
dT.plot.bar(y='count')
plt.title("Bar Plot of the Count of Numeric  features",fontsize=17)
plt.show()

In [ ]:
train.drop('Cabin',axis=1,inplace=True)
train.dropna(inplace=True)
train.head

In [ ]:
train.drop(['PassengerId','Name','Ticket'],axis=1,inplace=True)
train.dropna(inplace=True)
train.head()

In [ ]:
sex=pd.get_dummies(train ['Sex'],drop_first=True)
embark=pd.get_dummies(train['Embarked'],drop_first=True)

In [ ]:
train.drop(['Sex','Embarked'],axis=1,inplace=True)
train=pd.concat([train,sex,embark],axis=1)
train.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(
    train.drop('Survived',axis=1),
    train['Survived'],
    test_size=0.30,
    random_state=111
)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
nsimu = 201
penalty = [0] * nsimu
logmodel = [0] * nsimu
predictions = [0] * nsimu
class_report = [0] * nsimu
f1 = [0] * nsimu
for i in range(1, nsimu):
  logmodel[i] = LogisticRegression(C=i/1000, tol=1e-4, max_iter=int(1e6), n_jobs =4)
  logmodel[i].fit(X_train, y_train)
  predictions[i] = logmodel[i].predict(X_test)
  class_report[i] = classification_report(y_test, predictions[i])
  l = class_report[i].split()
  f1[i] = l[len(l)-2]
  penalty[i] = 1000/i
plt.scatter(penalty[1:len(penalty) - 2], f1[1:len(f1) - 2])
plt.title("F1-score vs. Regularization Parameter", fontsize=20)
plt.xlabel("Penalty Parameter", fontsize=17)
plt.ylabel("F1-score on Test Data", fontsize=17)
plt.show()